In [1]:
import pandas as pd
import numpy as np
from python_speech_features import mfcc
import scipy.io.wavfile as wav

from tempfile import TemporaryFile

import os
import pickle
import random
import operator

import math


In [2]:
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k) + distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))

    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    
    return neighbors

In [3]:
# identify the class of the instance
def nearestClass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)

    return sorter[0][0]

In [4]:
def getAccuracy(testSet, prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    
    return (1.0 * correct) / len(testSet)

In [5]:
directory = "/home/rajab/Documents/Dataku/Dokument/Data Rajab Agung/Data Kuliah/Mata Kuliah/Semester 5/Metopen/Music_Classification_forstudy/Data/genres_original2/"
f = open("my.dat", 'wb')

i = 0

for folder in os.listdir(directory):
    i += 1
    if i == 11:
        break
    for file in os.listdir(directory+folder):        
        try:
            (rate, sig) = wav.read(directory+folder+"/"+file)
            mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            print('Got an exception: ', e, ' in folder: ', folder, ' filename: ', file)        

f.close()

In [6]:
dataset = []
def loadDataset(filename, split,trSet, teSet):
    with open("my.dat",'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break

    for x in range(len(dataset)):
        if random.random() < split:
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])

trainingSet = []
testSet = []

loadDataset("my.dat", 0.70, trainingSet, testSet)

In [7]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

In [8]:
leng = len(testSet)
predictions = []
for x in range(leng):
    predictions.append(nearestClass(getNeighbors(trainingSet, testSet[x], 5)))

accuracy1 = getAccuracy(testSet, predictions)
print(accuracy1)
print(predictions)

0.7138263665594855
[10, 1, 1, 1, 6, 1, 6, 1, 1, 3, 1, 1, 3, 1, 8, 1, 1, 1, 1, 10, 1, 1, 1, 7, 6, 10, 1, 1, 1, 1, 10, 1, 2, 2, 10, 2, 2, 7, 2, 2, 2, 6, 2, 10, 1, 6, 6, 3, 2, 2, 6, 10, 6, 6, 6, 2, 2, 6, 2, 2, 10, 2, 10, 2, 2, 2, 2, 10, 3, 10, 3, 3, 3, 3, 10, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 1, 3, 3, 3, 7, 4, 4, 4, 4, 4, 4, 6, 4, 4, 4, 4, 4, 4, 4, 4, 7, 7, 4, 4, 10, 4, 4, 4, 4, 10, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 10, 5, 9, 5, 5, 4, 5, 5, 5, 2, 7, 4, 1, 5, 9, 2, 5, 7, 3, 10, 5, 5, 5, 7, 9, 5, 7, 5, 5, 5, 9, 5, 5, 5, 5, 5, 6, 6, 3, 6, 6, 6, 6, 6, 10, 1, 2, 6, 6, 6, 6, 6, 3, 6, 7, 1, 6, 6, 6, 6, 6, 1, 6, 6, 10, 7, 6, 10, 6, 7, 7, 10, 6, 2, 10, 7, 2, 2, 4, 7, 7, 7, 7, 7, 5, 7, 10, 7, 7, 7, 7, 1, 7, 7, 7, 7, 7, 7, 2, 7, 7, 2, 1, 8, 8, 8, 8, 8, 8, 8, 8, 6, 2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 5, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 7, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 3, 1, 10, 10, 1, 10, 10, 10, 10, 1, 10, 10, 7, 6, 10, 10, 10, 7, 10, 10, 10]


In [9]:
def is_suitable_for_studying(predictions):
    # Define a threshold for suitability based on your data
    threshold = 4  # Adjust this threshold as needed
    return predictions >= threshold

In [10]:
import tkinter as tk
from tkinter import filedialog
import customtkinter as sr
import pygame 

sr.set_appearance_mode("light")
sr.set_default_color_theme("dark-blue")

root = sr.CTk()
root.title("Seleksi Genre Lagu")
root.geometry("800x500")

def initialize_mixer():
    pygame.mixer.init()

initialize_mixer()

# def 
def generate():
    file_path = entry.get()
    if not file_path:
        outcomeLabel.configure(text="Lagu belum dipilih")
        return
    if not file_path.endswith(".wav"):
        outcomeLabel.configure(text="Dimohon untuk memasukkan file WAV.")
        return
        
    test_file = entry.get()
    (rate, sig) = wav.read(test_file)
    mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
    covariance = np.cov(np.matrix.transpose(mfcc_feat))
    mean_matrix = mfcc_feat.mean(0)

    from collections import defaultdict
    results = defaultdict(int)

    directory = "/home/rajab/Documents/Dataku/Dokument/Data Rajab Agung/Data Kuliah/Mata Kuliah/Semester 5/Metopen/Music_Classification_forstudy/Data/genres_original2/"

    i = 1
    for folder in os.listdir(directory):
        results[i] = folder
    i += 1

    feature = (mean_matrix, covariance, i)

    pred = nearestClass(getNeighbors(dataset, feature, 5))
    is_suitable = is_suitable_for_studying(pred)

    if is_suitable:
        # Hasil = ("Apakah lagu cocok untuk meningkatkan mood belajar? {}".format(is_suitable))
        outcomeLabel.configure(text="Lagu cocok untuk meningkatkan mood belajar.")
        return

    outcomeLabel.configure(text="Lagu kurang cocok untuk meningkatkan mood belajar.")
    # Hasil = ("Apakah lagu cocok untuk meningkatkan mood belajar? {}".format(is_suitable))
    # outcomeLabel.configure(text=Hasil)

def browse():
    file_path = filedialog.askopenfilename(filetypes=[("WAV files", "*.wav")])
    entry.delete(0, tk.END)
    entry.insert(0, file_path)\
    
def play():
    file_path = entry.get()
    if not file_path:
        outcomeLabel.configure(text="Lagu belum dipilih")
        return
    if not file_path.endswith(".wav"):
        outcomeLabel.configure(text="Dimohon untuk memasukkan file WAV.")
        return
    
    outcomeLabel.configure(text="")
    
    pygame.mixer.init()
    pygame.mixer.music.load(file_path)
    pygame.mixer.music.play()

def stop():
    pygame.mixer.music.stop()

def quit():
    stop()
    root.destroy()

#layout
frame = sr.CTkFrame(master=root)
frame.pack(pady=20, padx=60, fill="both", expand=True)

label = sr.CTkLabel(master=frame, text="Input Lagu")
label.pack(pady=12, padx=10)

entry = sr.CTkEntry(master=frame, placeholder_text="Masukkan lagunya", width=500)
entry.pack(pady=12, padx=10)

buttonFile = sr.CTkButton(master=frame, text="Browse File", command=browse)
buttonFile.pack(pady=20, padx=20)

buttonPlay = sr.CTkButton(master=frame, text="Play Song", command=play)
buttonPlay.pack(pady=12, padx=10)

buttonStop = sr.CTkButton(master=frame, text="Stop Song", command=stop)
buttonStop.pack(pady=12, padx=10)

button = sr.CTkButton(master=frame, text="Generate", command=generate )
button.pack(pady=12, padx=10)

outcomeLabel=sr.CTkLabel(master=frame, text="")
outcomeLabel.pack(pady=12, padx=10)

buttonOut = buttonFile = sr.CTkButton(master=frame, text="Keluar", command=quit)
buttonFile.pack(pady=20, padx=20)

root.mainloop()


pygame 2.5.2 (SDL 2.28.2, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [11]:

# test_file = entry.get()
# # test_file = test_dir + "test3.wav"
# # test_file = test_dir + "test3.wav"
# # test_file = test_dir + "test4.wav"
# # test_file = test_dir + "test5.wav"

In [12]:
# (rate, sig) = wav.read(test_file)
# mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
# covariance = np.cov(np.matrix.transpose(mfcc_feat))
# mean_matrix = mfcc_feat.mean(0)
# feature = (mean_matrix, covariance, i)

In [13]:
# from collections import defaultdict
# results = defaultdict(int)

# directory = "/home/rajab/Documents/Dataku/Dokument/Data Rajab Agung/Data Kuliah/Mata Kuliah/Semester 5/Metopen/Music_Classification_forstudy/Data/genres_original/"

# i = 1
# for folder in os.listdir(directory):
#     results[i] = folder
#     i += 1

# pred = nearestClass(getNeighbors(dataset, feature, 5))
# is_suitable = is_suitable_for_studying(pred)
# print("Is suitable for studying:", is_suitable)